# Data from mutiple sources

## Intro

Now that we have a handle on using Folium and entering data from our own source, we are going to start looking at incorporating data from multiple locations to produce a richer map. We are going to look at a few different areas we can draw data from, and how we can incorporate them into our maps:
- [GeoJSON](https://geojson.org/) files. In this case we will be looking at two things to do with coastlines:
    - Whether our paths cross them
    - Whether the ship is close enough to them to have made a sighting
- Weather data - and how to display it
- Tide data - (if I can find a good source) and how to display it

## Aims 

- Defining functions we can then use later


## Data

Simply Googling is an effective way to find access to GeoJson and shapefile data. Searching for a "world map shapefile", for example, is liley to lead you to [World Bank](https://datacatalog.worldbank.org/search/dataset/0038272/World-Bank-Official-Boundaries) data. From here, we can find a link to a datasource called "World coastlines", which we should download. 

Note that, as with many types of data, some areas of the world are better covered than others. You may also find data behind a pay or register wall, or with other restrictions. There are, however, plently of data sources - feel free to have a go at searching for your own. For the examples in this notebook, we are going to be using the coastline data from World Bank. 


## Loading required libraries

As with the previous notebooks, we need to load some libraries. Some of these are going to be familiar, but we are also going to be using some new ones - these will be explained as we go along.

## Creating a new map

(Creates a map of the UK, and shows the course of the Spanish Ladies song - this clearly overlaps the coast in a couple of locations)

## Investigating the GeoJSON file

If we open the 

## Finding overlaps (writing your own functions)

We are going to be defining our own function to find overlaps between our path and the coastline. Having this function makes it much easier to seperate out the legs of our ships journey and check each one individually.

For this funciton we are going to use the [Shapely](https://shapely.readthedocs.io/en/stable/manual.html) library. This is a library that allows us to do geometric operations on shapes. In this case, we are going to be using it to find the intersection between our path and the coastline. We can see the documentation for the intersection function [here](https://shapely.readthedocs.io/en/stable/reference/shapely.intersection.html). From this documentation, we can see that we need to pass in two shapes, and the funciton will return the coordiates where they intersect. As we don't actually care about the coordinates, we can just pass in the shapes and check if the result is empty or not.

## Making the overlaps show up on the map

Now we're able to tell which legs overlap, lets go back and adjust the data we're mapping to show this. Where we looped through and created a feature for each leg, we can now add a check to see if the leg overlaps the coastline. If it does, we can are going to change the colour of the line to red for that leg. 

## Sighting land

In the previous notebook, we discussed dead reckoning. One of the assumptions we made was that the ship never sighted land until the end of their journey, thus the error was compounding. In reality, when they sighted land, they could make an accurate fix to their location. Here, we are going look at how we can bring that data into our map.

#### How close 

Visibility of land depends on how high you are above the water. This is known as the dipping distance, and the higher you are, the further you can see. While we have good information on some ships, and the height of their crows nest, for this notebook we are going to assume a 20 nautical mile visibility. This is a reasonable assumption for a ship of the time, and is also a nice round number.

## And now, over to you

Here we're just showing an example on how to highlight 